# Test problem from Caroe et al. (1999)

$$
\begin{align*}
\max_{x}      \quad & \frac{3}{2} x_1 + 4 x_2 + Q(x_1, x_2) \\
\mathrm{s.t.} \quad & 0 \leq x \leq 5 \\
                    & x \ \mathrm{integer}
\end{align*}
$$

where $Q(x_1,x_2)$:
$$
\begin{align*}
\max_{y}      \quad & 16 y_1 + 19 y_2 + 23 y_3 + 28 y_4 \\
\mathrm{s.t.} \quad & 2 y_1 + 3 y_2 + 4 y_3 + 5 y_4 \leq \zeta_1 - x_1 \\
                    & 6 y_1 + y_2 + 3 y_3 + 2 y_4 \leq \zeta_2 - x_2 \\
                    & y_i \in \{0,1\}
\end{align*}
$$
and $\zeta = (\zeta_1,\zeta_2)$ is uniformly distributed on
$$\Pi = \{(5,5), (5,6),\dots,(5,15),(6,5),\dots,(15,15)\}$$
giving 121 scenarios.

In [1]:
RELAX_LP = True

In [2]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

import numpy as np
import pandas as pd

In [3]:
zetas = [(i,j) for i in np.arange(5,16) for j in np.arange(5,16)]
print('Scenarios: %s'%len(zetas))

Scenarios: 121


In [4]:
cx = -np.array([3/2, 4])
fy = -np.array([16, 19, 23, 28])
xl = [0, 0]
xu = [5, 5]
A  = [[2,3,4,5],
      [6,1,3,2]]

sub_dict = {}
for k,zeta in enumerate(zetas):
    mdl = Model('sub')    
    ys  = [Variable('y_%d'%j, lower_bound=0, upper_bound=1, objective_coefficient=fy[j]) for j in range(len(fy))]
    xs  = [Variable('x_%d'%j, lower_bound=xl[j], upper_bound=xu[j], objective_coefficient=cx[j]) for j in range(len(cx))]
    for x in xs:
        x.variable_kind = 'integer'
    for y in ys:
        y.variable_kind = 'integer'
    mdl.add_reactions(xs+ys)
    for i,ai in enumerate(A):
        cons = Constraint('cons_%d'%i)
        cons._constraint_sense = 'L'
        cons._bound = zeta[i]
        xs[i].add_metabolites({cons:1})
        for j,aij in enumerate(ai):
            ys[j].add_metabolites({cons:aij})
    sub_id = 'scen_%d'%k
    sub = LagrangeSubmodel(mdl, sub_id, first_stage_vars=xs)
    sub._weight = 1./len(zetas)
    sub_dict[sub_id] = sub

In [5]:
master = LagrangeMaster(mdl, first_stage_vars=xs)
master._INF = 1e8
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF
master.model.Params.Presolve = 0
master.model.update()
master.model.Params.NodefileStart = 3.
for sub in sub_dict.values():
    sub.model.Params.NodefileStart = 3.

In [6]:
master.gaptol = 1e-3
master.penaltytol = 1e-6 #1e-4
master.delta_mult = 0.5

feasible_methods = ['heuristic','enumerate']
alt_method = 'pool'
max_alt = 10


if RELAX_LP:
    sol_master = master.solve_relaxed(feasible_methods=None)
    df_phase1 = pd.DataFrame(master.log_rows)

master.gaptol = 0.01
master.verbosity=2
sol_master = master.optimize(feasible_methods=feasible_methods, max_alt=max_alt, alt_method=alt_method) 
df_phase2 = pd.DataFrame(master.log_rows)

    Iter                    UB                    LB       gap relgap(%)   penalty                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Best   Feasible        Sub       Best                                   total    master       sub
       0      1e+08     1e+100     -73.54     -73.54     1e+08       100         0  0.077361  0.003963  0.068830
      10     -66.63     1e+100     -68.03     -68.03     1.394     2.092     0.368  0.534734  0.015243  0.037804
      15      -66.9     1e+100      -66.9      -66.9         0         0  1.93e-13  0.885039  0.041174  0.037391
    Iter                    UB                    LB       gap relgap(%)   penalty                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Best   Feasible        Sub       Best                                   total    

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition.py:2149 UserWarning: Solution available but Master solver status=suboptimal (13).


Master solver status=suboptimal (13).
     100     -67.01     1e+100     -65.55     -65.55     1.465     2.187  4.29e-10  71.63590  1.702749  0.091627
     110      -66.9     1e+100     -65.59     -65.55     1.352     2.021  2.65e-11  92.02553  1.800462  0.090672
     120      -66.9     1e+100     -65.59     -65.55     1.352     2.021  4.29e-12  118.5945  2.291068  0.090750
     130      -66.9     1e+100     -65.59     -65.55     1.352     2.021  5.77e-12  152.1065  4.775872  0.092296
     140      -66.9     1e+100     -65.59     -65.55     1.352     2.021  1.03e-12  191.6328  3.719273  0.092434
Master solver status=suboptimal (13).
     150     -67.28     1e+100      -65.5      -65.5     1.773     2.635  3.39e-09  236.4122  4.010673  0.096095
     160      -66.9     1e+100     -65.59      -65.5     1.395     2.085  9.76e-12  285.8998  5.636308  0.090898
Master solver status=suboptimal (13).
     170      -66.9     1e+100     -65.59      -65.5     1.395     2.085  3.26e-11  343.1572  5

KeyError: 11

In [7]:
sub.model.getObjective()

<gurobi.LinExpr: 1.32724432647 x_0 + -2.66666717827 x_1 + -16.0 y_0 + -19.0 y_1 + -23.0 y_2 + -28.0 y_3>

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

xcol = 'iter'
df_phase1.loc[:,'phase'] = 1
df_phase1.loc[:,'feasUB'] = np.nan
df_phase2.loc[:,'phase'] = 2
dsplot = pd.concat([df_phase1, df_phase2])
dsplot = dsplot[[xcol,'feasUB','bestUB','bestLB','phase']]
dsplot = dsplot.melt(id_vars=[xcol,'phase'])

g = sns.FacetGrid(dsplot, hue='variable', size=3, aspect=3, row='phase', sharey=False)
g.map(plt.plot, xcol,'value')
g.add_legend()

NameError: name 'df_phase2' is not defined